# Requierments


In [ ]:
%pip install sentence_transformers
%pip install hazm
! python -m pip install statsmodels

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 KB 4.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 41.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 39.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 KB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 20.6 MB/s eta 0:00:00
  Created wheel for sentence_transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125938 sha256=6e2f63e77144b658680bc43592efcdade4916baee12c63142773d78ffc672cfd
  Stored in directory: /root/.cache/pip/wheels/71/67/06/162a3760c40d74dd40bc855d527008d26341c2b0ecf3e8e11f
Successfully built sentence_transformers
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
from sentence_transformers import SentenceTransformer
import pandas as pd
import random as rd
import json
import numpy as np
import re
import numpy as np
import nltk
import string
from statsmodels.stats.inter_rater import fleiss_kappa
from sklearn.metrics import ndcg_score

# Aggregate Data After Tagging

In [ ]:
tagged11 =pd.read_csv("/content/1 - 1.csv")
tagged12 =pd.read_csv("/content/new_1_tagged.csv")
tagged21 =pd.read_csv("/content/2 _tagged.csv")
tagged22 =pd.read_csv("/content/new_2_tagged.csv")
tagged3 =pd.read_csv("/content/new3_tagged.csv")
tagged41 =pd.read_csv("/content/4_1.csv")
tagged42 =pd.read_csv("/content/4_2.csv")

In [ ]:
tagged11 = tagged11.drop(['Unnamed: 0', 'index'], axis=1)
tagged11

,input_id,output_id,input_text,output_text,chapter,tag
0,46,2987,نباید عیب دیگران را بر ملا کنیم,بی قدر ساخت خود را، نخوت فزود ما را بر ما و خو...,NaN,-1
1,15,3128,کنترل نکردن زبان انسان را بی‌ارزش می‌کند.,بی‌جوهران به تربیت آدم نمی‌شوند شبنم به بوی گل...,NaN,0
2,15,3353,کنترل نکردن زبان انسان را بی‌ارزش می‌کند.,I love mankind ... it's people I can't stand!,NaN,0
3,7,2922,ثروت چیز خوبی است,تواضع گر چه محبوب است و فضل بیکران دارد نباید ...,NaN,0
4,30,3203,نباید مغرور باشیم,در این عالم از هرچه بایستنیست نکوتر ز فرزند شا...,NaN,0
...,...,...,...,...,...,...
67,33,3406,نباید در خوردن زیاده‌روی کرد,You can never be overdressed or overeducated,NaN,0
68,36,3172,باید اهل یاری کردن یکدیگر بود,چون پرده براندازد عالم به سرانداز جایی که یقین...,NaN,0
69,3,5310,باید خاک وطن را دوست داشت,و قال ع فقد الاحبه غربه,65,0
70,16,5592,نباید از دیگران درخواست های متعدد داشت,و قال ع من طلب شیئا ناله او بعضه,386,0


In [ ]:
tagged12 = tagged12.drop(['Unnamed: 0', 'Unnamed: 0.1'], axis=1)
tagged12

,input_id,output_id,input_text,output_text,chapter,tag
0,49,3858,لزوما رسیدن به چیزی که آرزو داریم به نفع ما نیست,Life's under no obligation to give us what we ...,NaN,2
1,1,3181,ادب یکی از صفات اخلاقی نیکو است که انسان باید ...,حاصل زندگی جز این نبود که بمیرد کسی برای کسی,NaN,0
2,3,3191,باید خاک وطن را دوست داشت,خاکم به سر ز غصه به سر خاک اگر کنم خاک وطن که ...,NaN,2
3,4,2956,نباید براساس ظاهر افراد را قضاوت کرد,نیست نشان زندگی تا نرسد نشان تو,NaN,0
4,4,3049,نباید براساس ظاهر افراد را قضاوت کرد,آدمی باید که بی‌حالی نباشد هیچگاه گر لب خندان ...,NaN,0
...,...,...,...,...,...,...
673,49,2973,لزوما رسیدن به چیزی که آرزو داریم به نفع ما نیست,رسد آدمی به جایی که به جز خدا نبیند بنگر که تا...,NaN,2
674,48,1692,باید از منافق دوری کنیم,بار دیگر از دل و از عقل و جان برخاستیم یار آمد...,1601,1
675,12,3690,انسان واقعی در برابر ظلم سکوت نمی‌کند,"For you, a thousand times ove",NaN,0
676,48,403,باید از منافق دوری کنیم,می‌فکن بر صف رندان نظری بهتر از این بر در میکد...,غزل ۴۰۴,0


In [ ]:
tagged21 = tagged21.drop(['Unnamed: 0', 'index'], axis=1)
tagged21

,input_id,output_id,input_text,output_text,chapter,tag
0,21,5207,آخرت را بر دنیا باید ترجیح داد,No one is useless in this world who lightens t...,NaN,1
1,21,2506,آخرت را بر دنیا باید ترجیح داد,ای بی تو حرام زندگانی خود بی تو کدام زندگانی ب...,2734,0
2,21,1205,آخرت را بر دنیا باید ترجیح داد,دیده ها شب فراز باید کرد روز شد دیده باز باید ...,970,1
3,21,124,آخرت را بر دنیا باید ترجیح داد,شاهد آن نیست که مویی و میانی دارد بنده طلعت آن...,غزل ۱۲۵,0
4,21,303,آخرت را بر دنیا باید ترجیح داد,دارای جهان نصرت دین خسرو کامل یحیی بن مظفر ملک...,غزل ۳۰۴,0
...,...,...,...,...,...,...
645,47,4968,هرکاری را می‌توان انجام داد به جز تغییر عادت‌ها,Those who cannot change their minds cannot cha...,NaN,1
646,47,3114,هرکاری را می‌توان انجام داد به جز تغییر عادت‌ها,بجز بنای محبت که دائم آباد است خراب می‌کند ایا...,NaN,0
647,47,3446,هرکاری را می‌توان انجام داد به جز تغییر عادت‌ها,"Everyone thinks of changing the world, but no ...",NaN,1
648,47,238,هرکاری را می‌توان انجام داد به جز تغییر عادت‌ها,رسید مژده که آمد بهار و سبزه دمید وظیفه گر برس...,غزل ۲۳۹,0


In [ ]:
tagged22 = tagged22.drop(['Unnamed: 0', 'Unnamed: 0.1'], axis=1)
tagged22

,input_id,output_id,input_text,output_text,chapter,tag
0,1,5442,ادب یکی از صفات اخلاقی نیکو است که انسان باید ...,و قال ع الخلاف یهدم الرای‌,215,0
1,1,3218,ادب یکی از صفات اخلاقی نیکو است که انسان باید ...,در این عالم از هرچه بایستنیست نکوتر ز فرزند شا...,NaN,0
2,26,4956,انسان نباید فخر فروشی داشته باشد.,Don't cry over someone who wouldn't cry over you,NaN,0
3,26,3912,انسان نباید فخر فروشی داشته باشد.,God has no religion,NaN,0
4,12,3690,انسان واقعی در برابر ظلم سکوت نمی‌کند,"For you, a thousand times ove",NaN,0
...,...,...,...,...,...,...
107,30,336,نباید مغرور باشیم,چرا نه در پی عزم دیار خود باشم چرا نه خاک سر ک...,غزل ۳۳۷,0
108,30,4727,نباید مغرور باشیم,If we couldn't laugh we would all go insane,NaN,0
109,30,3784,نباید مغرور باشیم,Being crazy isn't enough,NaN,0
110,47,5,هرکاری را می‌توان انجام داد به جز تغییر عادت‌ها,به ملازمان سلطان که رساند این دعا را که به شکر...,غزل ۶,0


In [ ]:
tagged3 = tagged3.drop(['Unnamed: 0', 'Unnamed: 0.1'], axis=1)
tagged3

,input_id,output_id,input_text,output_text,chapter,tag
0,0,5318,باید حقوق دیگران را رعایت کنیم,و قال ع من نصب نفسه للناس اماما فلیبدا بتعلیم ...,73,1
1,4,5564,نباید براساس ظاهر افراد را قضاوت کرد,و قال ع للظالم من الرجال ثلاث علامات یظلم من ف...,350,0
2,9,5563,باید قدردان دارایی‌های خودمان باشیم,و قال ع اشد الذنوب ما استهان به صاحبه,348,0
3,13,5563,باید بخشنده باشیم,و قال ع اشد الذنوب ما استهان به صاحبه,348,0
4,14,5414,باید حق‌جو وعدالت‌پیشه باشیم,و قال ع ما شککت فی الحق مذ اریته,184,2
...,...,...,...,...,...,...
705,24,3396,باید در برابر مشکلات استقامت داشته باشیم,A day without laughter is a day wasted,NaN,0
706,5,618,باید تجارب خود را به افراد دیگر منتقل کنیم,از سینه پاک کردم افکار فلسفی را در دیده جای کر...,187,0
707,47,3851,هرکاری را می‌توان انجام داد به جز تغییر عادت‌ها,"Wow,"" Thalia muttered. ""Apollo is hot."" ""He's ...",NaN,0
708,2,9,ما باید از غیبت کردن پرهیز کنیم,دوش از مسجد سوی میخانه آمد پیر ما چیست یاران ط...,غزل ۱۰,0


In [ ]:
tagged41 = tagged41.drop(['Unnamed: 0', 'Unnamed: 0.1'], axis=1)
tagged41

,input_id,output_id,input_text,output_text,chapter,tag
0,37,240,باید در هنگام پیمان بستن دقت کنیم,معاشران ز حریف شبانه یاد آرید حقوق بندگی مخلصا...,غزل ۲۴۱,1
1,2,5556,ما باید از غیبت کردن پرهیز کنیم,و قال ع ترک الذنب اهون من طلب المعونه,341,2
2,18,322,نباید اندوهگین شد,ز دست کوته خود زیر بارم که از بالابلندان شرمسا...,غزل ۳۲۳,2
3,37,2507,باید در هنگام پیمان بستن دقت کنیم,چون سوی برادری بپویی باید که نخست رو بشویی در ...,2736,2
4,46,5551,نباید عیب دیگران را بر ملا کنیم,و قال ع لا طاعه لمخلوق فی معصیه الخالق,336,1
...,...,...,...,...,...,...
573,34,4145,باید به قلب و احساسات توجه کرد,"One love, one heart, one destiny",NaN,1
574,23,5605,باید فرزندان خود را به خوبی تربیت کنیم,NaN,399,0
575,0,3181,باید حقوق دیگران را رعایت کنیم,حاصل زندگی جز این نبود که بمیرد کسی برای کسی,NaN,1
576,27,377,در کار خیر باید شتاب کرد و بدی را به تاخیر اند...,ما نگوییم بد و میل به ناحق نکنیم جامه کس سیه و...,غزل ۳۷۸,1


In [ ]:
tagged42 = tagged42.drop(['Unnamed: 0', 'Unnamed: 0.1','Unnamed: 8'], axis=1)
tagged42

,input_id,output_id,input_text,output_text,chapter,tag
0,0,5318,باید حقوق دیگران را رعایت کنیم,و قال ع من نصب نفسه للناس اماما فلیبدا بتعلیم ...,73,1
1,4,5564,نباید براساس ظاهر افراد را قضاوت کرد,و قال ع للظالم من الرجال ثلاث علامات یظلم من ف...,350,0
2,9,5563,باید قدردان دارایی‌های خودمان باشیم,و قال ع اشد الذنوب ما استهان به صاحبه,348,0
3,13,5563,باید بخشنده باشیم,و قال ع اشد الذنوب ما استهان به صاحبه,348,0
4,14,5414,باید حق‌جو وعدالت‌پیشه باشیم,و قال ع ما شککت فی الحق مذ اریته,184,1
...,...,...,...,...,...,...
95,16,3406,نباید از دیگران درخواست های متعدد داشت,You can never be overdressed or overeducated,NaN,1
96,19,4395,باید اهل یادگیری و دانش باشیم,"Children must be taught how to think, not what...",NaN,1
97,20,5162,باید مهربان بود,I just want to be wonderful,NaN,0
98,21,4207,آخرت را بر دنیا باید ترجیح داد,The world was hers for the reading,NaN,0


In [ ]:
frames = [tagged11,tagged12,tagged21,tagged22,tagged3,tagged41,tagged42]
all_tagged = pd.concat(frames)

In [ ]:
all_tagged

,input_id,output_id,input_text,output_text,chapter,tag
0,46,2987,نباید عیب دیگران را بر ملا کنیم,بی قدر ساخت خود را، نخوت فزود ما را بر ما و خو...,NaN,-1
1,15,3128,کنترل نکردن زبان انسان را بی‌ارزش می‌کند.,بی‌جوهران به تربیت آدم نمی‌شوند شبنم به بوی گل...,NaN,0
2,15,3353,کنترل نکردن زبان انسان را بی‌ارزش می‌کند.,I love mankind ... it's people I can't stand!,NaN,0
3,7,2922,ثروت چیز خوبی است,تواضع گر چه محبوب است و فضل بیکران دارد نباید ...,NaN,0
4,30,3203,نباید مغرور باشیم,در این عالم از هرچه بایستنیست نکوتر ز فرزند شا...,NaN,0
...,...,...,...,...,...,...
95,16,3406,نباید از دیگران درخواست های متعدد داشت,You can never be overdressed or overeducated,NaN,1
96,19,4395,باید اهل یادگیری و دانش باشیم,"Children must be taught how to think, not what...",NaN,1
97,20,5162,باید مهربان بود,I just want to be wonderful,NaN,0
98,21,4207,آخرت را بر دنیا باید ترجیح داد,The world was hers for the reading,NaN,0


In [ ]:
no_check =pd.read_csv("/content/no_check.csv")
first_check =pd.read_csv("/content/first_check.csv")
double_check =pd.read_csv("/content/double_check.csv")

In [ ]:
no_check = no_check.drop(['Unnamed: 0','tag'], axis=1)
first_check = first_check.drop(['Unnamed: 0','tag'], axis=1)
double_check = double_check.drop(['Unnamed: 0','tag'], axis=1)

In [ ]:
no_check.insert(loc = 5 , column = "tag1" , value = 0 )
no_check.insert(loc = 6 , column = "tag2" , value = 0 )
first_check.insert(loc = 5 , column = "tag1" , value = 0 )
first_check.insert(loc = 6 , column = "tag2" , value = 0 )
double_check.insert(loc = 5 , column = "tag1" , value = 0 )
double_check.insert(loc = 6 , column = "tag2" , value = 0 )

In [ ]:
double_check

,input_id,output_id,input_text,output_text,chapter,tag1,tag2
0,46,2987,نباید عیب دیگران را بر ملا کنیم,بی قدر ساخت خود را، نخوت فزود ما را بر ما و خو...,NaN,0,0
1,15,3128,کنترل نکردن زبان انسان را بی‌ارزش می‌کند.,بی‌جوهران به تربیت آدم نمی‌شوند شبنم به بوی گل...,NaN,0,0
2,15,3353,کنترل نکردن زبان انسان را بی‌ارزش می‌کند.,I love mankind ... it's people I can't stand!,NaN,0,0
3,7,2922,ثروت چیز خوبی است,تواضع گر چه محبوب است و فضل بیکران دارد نباید ...,NaN,0,0
4,30,3203,نباید مغرور باشیم,در این عالم از هرچه بایستنیست نکوتر ز فرزند شا...,NaN,0,0
...,...,...,...,...,...,...,...
67,33,3406,نباید در خوردن زیاده‌روی کرد,You can never be overdressed or overeducated,NaN,0,0
68,36,3172,باید اهل یاری کردن یکدیگر بود,چون پرده براندازد عالم به سرانداز جایی که یقین...,NaN,0,0
69,3,5310,باید خاک وطن را دوست داشت,و قال ع فقد الاحبه غربه,65,0,0
70,16,5592,نباید از دیگران درخواست های متعدد داشت,و قال ع من طلب شیئا ناله او بعضه,386,0,0


In [ ]:
first_check

,input_id,output_id,input_text,output_text,chapter,tag1,tag2
0,37,240,باید در هنگام پیمان بستن دقت کنیم,معاشران ز حریف شبانه یاد آرید حقوق بندگی مخلصا...,غزل ۲۴۱,0,0
1,2,5556,ما باید از غیبت کردن پرهیز کنیم,و قال ع ترک الذنب اهون من طلب المعونه,341,0,0
2,18,322,نباید اندوهگین شد,ز دست کوته خود زیر بارم که از بالابلندان شرمسا...,غزل ۳۲۳,0,0
3,37,2507,باید در هنگام پیمان بستن دقت کنیم,چون سوی برادری بپویی باید که نخست رو بشویی در ...,2736,0,0
4,46,5551,نباید عیب دیگران را بر ملا کنیم,و قال ع لا طاعه لمخلوق فی معصیه الخالق,336,0,0
...,...,...,...,...,...,...,...
573,34,4145,باید به قلب و احساسات توجه کرد,"One love, one heart, one destiny",NaN,0,0
574,23,5605,باید فرزندان خود را به خوبی تربیت کنیم,و قال ع ان للولد علی الوالد حقا و ان للوالد عل...,399,0,0
575,0,3181,باید حقوق دیگران را رعایت کنیم,حاصل زندگی جز این نبود که بمیرد کسی برای کسی,NaN,0,0
576,27,377,در کار خیر باید شتاب کرد و بدی را به تاخیر اند...,ما نگوییم بد و میل به ناحق نکنیم جامه کس سیه و...,غزل ۳۷۸,0,0


In [ ]:
no_check

,input_id,output_id,input_text,output_text,chapter,tag1,tag2
0,0,5318,باید حقوق دیگران را رعایت کنیم,و قال ع من نصب نفسه للناس اماما فلیبدا بتعلیم ...,73,0,0
1,4,5564,نباید براساس ظاهر افراد را قضاوت کرد,و قال ع للظالم من الرجال ثلاث علامات یظلم من ف...,350,0,0
2,9,5563,باید قدردان دارایی‌های خودمان باشیم,و قال ع اشد الذنوب ما استهان به صاحبه,348,0,0
3,13,5563,باید بخشنده باشیم,و قال ع اشد الذنوب ما استهان به صاحبه,348,0,0
4,14,5414,باید حق‌جو وعدالت‌پیشه باشیم,و قال ع ما شککت فی الحق مذ اریته,184,0,0
...,...,...,...,...,...,...,...
795,49,2973,لزوما رسیدن به چیزی که آرزو داریم به نفع ما نیست,رسد آدمی به جایی که به جز خدا نبیند بنگر که تا...,NaN,0,0
796,48,1692,باید از منافق دوری کنیم,بار دیگر از دل و از عقل و جان برخاستیم یار آمد...,1601,0,0
797,12,3690,انسان واقعی در برابر ظلم سکوت نمی‌کند,"For you, a thousand times ove",NaN,0,0
798,48,403,باید از منافق دوری کنیم,می‌فکن بر صف رندان نظری بهتر از این بر در میکد...,غزل ۴۰۴,0,0


In [ ]:
frames = [no_check,first_check,double_check]
all_pair = pd.concat(frames)

In [ ]:
all_pair

,input_id,output_id,input_text,output_text,chapter,tag1,tag2
0,0,5318,باید حقوق دیگران را رعایت کنیم,و قال ع من نصب نفسه للناس اماما فلیبدا بتعلیم ...,73,0,0
1,4,5564,نباید براساس ظاهر افراد را قضاوت کرد,و قال ع للظالم من الرجال ثلاث علامات یظلم من ف...,350,0,0
2,9,5563,باید قدردان دارایی‌های خودمان باشیم,و قال ع اشد الذنوب ما استهان به صاحبه,348,0,0
3,13,5563,باید بخشنده باشیم,و قال ع اشد الذنوب ما استهان به صاحبه,348,0,0
4,14,5414,باید حق‌جو وعدالت‌پیشه باشیم,و قال ع ما شککت فی الحق مذ اریته,184,0,0
...,...,...,...,...,...,...,...
67,33,3406,نباید در خوردن زیاده‌روی کرد,You can never be overdressed or overeducated,NaN,0,0
68,36,3172,باید اهل یاری کردن یکدیگر بود,چون پرده براندازد عالم به سرانداز جایی که یقین...,NaN,0,0
69,3,5310,باید خاک وطن را دوست داشت,و قال ع فقد الاحبه غربه,65,0,0
70,16,5592,نباید از دیگران درخواست های متعدد داشت,و قال ع من طلب شیئا ناله او بعضه,386,0,0


In [ ]:
all_pair.insert(loc = 7 , column = "-2" , value = 0 )
all_pair.insert(loc = 8 , column = "-1" , value = 0 )
all_pair.insert(loc = 9 , column = "0" , value = 0 )
all_pair.insert(loc = 10 , column = "1" , value = 0 )
all_pair.insert(loc = 11 , column = "2" , value = 0 )


In [ ]:
all_pair

,input_id,output_id,input_text,output_text,chapter,tag1,tag2,-2,-1,0,1,2
0,0.0,5318.0,باید حقوق دیگران را رعایت کنیم,و قال ع من نصب نفسه للناس اماما فلیبدا بتعلیم ...,73,1.0,1.0,0.0,0.0,0.0,2.0,0.0
1,4.0,5564.0,نباید براساس ظاهر افراد را قضاوت کرد,و قال ع للظالم من الرجال ثلاث علامات یظلم من ف...,350,0.0,0.0,0.0,0.0,2.0,0.0,0.0
2,9.0,5563.0,باید قدردان دارایی‌های خودمان باشیم,و قال ع اشد الذنوب ما استهان به صاحبه,348,0.0,0.0,0.0,0.0,2.0,0.0,0.0
3,13.0,5563.0,باید بخشنده باشیم,و قال ع اشد الذنوب ما استهان به صاحبه,348,0.0,0.0,0.0,0.0,2.0,0.0,0.0
4,14.0,5414.0,باید حق‌جو وعدالت‌پیشه باشیم,و قال ع ما شککت فی الحق مذ اریته,184,2.0,1.0,0.0,0.0,0.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...
971,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN
972,NaN,NaN,NaN,NaN,NaN,0.0,1.0,NaN,NaN,NaN,NaN,NaN
973,NaN,NaN,NaN,NaN,NaN,-1.0,1.0,NaN,NaN,NaN,NaN,NaN
974,NaN,NaN,NaN,NaN,NaN,1.0,1.0,NaN,NaN,NaN,NaN,NaN


In [ ]:
columns = ["input_id","output_id","input_text","output_text","chapter","tag1","tag2","-2","-1","0","1","2"]
finall_output = pd.DataFrame([], columns=columns)

In [ ]:
for i in range(len(all_pair)):
  input_id_2 = int(all_pair.iloc[i]['input_id'])
  output_id_2 = int(all_pair.iloc[i]['output_id'])
  flag_1 = False
  counter = 0
  print(i)
  for j in range(len(all_tagged)):
    
    input_id = int(all_tagged.iloc[j]['input_id'])
    output_id = int(all_tagged.iloc[j]['output_id'])
    if(input_id == input_id_2 and output_id == output_id_2):
      value = int(all_tagged.iloc[j]['tag'])
      if(flag_1):
        # print("**** start *****")
        # print(int(all_tagged.iloc[j]['tag']))
        tag2 = value
        # print(all_pair.iloc[i]['tag2'])
        # print("**** end *****")
      else: 
        flag_1 = True
        # print("**** start *****")

        # print(int(all_tagged.iloc[j]['tag']))
        tag1 = value
        # print(all_pair.iloc[i]['tag1'])

        # print("**** end *****")
      counter += 1
  # print(counter)
  # tag1 = int(all_pair.iloc[i]['tag1'])
  # tag2 = int(all_pair.iloc[i]['tag2'])
  
  m2 = 0
  m1 = 0
  z0 = 0
  p1 = 0
  p2 = 0
  if(tag1 == -2):
    m2 += 1
  elif(tag1 == -1):
    m1+=1
  elif(tag1 == 0):
    z0+=1
  elif(tag1 == 1):
    p1+=1
  elif(tag1 == 2):
    p2+=1
  if(tag2 == -2):
    m2 += 1
  elif(tag2 == -1):
    m1+=1
  elif(tag2 == 0):
    z0+=1
  elif(tag2 == 1):
    p1+=1
  elif(tag2 == 2):
    p2+=1

  finall_output = finall_output.append({'input_id':all_pair.iloc[i]["input_id"],'output_id':all_pair.iloc[i]["output_id"],'input_text':all_pair.iloc[i]["input_text"],'output_text':all_pair.iloc[i]["output_text"],'chapter':all_pair.iloc[i]["chapter"],'tag1':tag1,'tag2':tag2,"-2":m2,"-1":m1,"0":z0,"1":p1,"2":p2}, ignore_index = True)
  # all_pair.at[i,str(tag1)] += 1
  # all_pair.at[i,str(tag2)] += 1
  print("len is " + str(len(finall_output)))
  if(counter > 2 or counter < 2 ):
    print("bad condition " + str(i))
    

0
len is 1
1
len is 2
2
len is 3
3
len is 4
4
len is 5
5
len is 6
6
len is 7
7
len is 8
8
len is 9
9
len is 10
10
len is 11
11
len is 12
12
len is 13
13
len is 14
14
len is 15
15
len is 16
16
len is 17
17
len is 18
18
len is 19
19
len is 20
20
len is 21
21
len is 22
22
len is 23
23
len is 24
24
len is 25
25
len is 26
26
len is 27
27
len is 28
28
len is 29
29
len is 30
30
len is 31
31
len is 32
32
len is 33
33
len is 34
34
len is 35
35
len is 36
36
len is 37
37
len is 38
38
len is 39
39
len is 40
40
len is 41
41
len is 42
42
len is 43
43
len is 44
44
len is 45
45
len is 46
46
len is 47
47
len is 48
48
len is 49
49
len is 50
50
len is 51
51
len is 52
52
len is 53
53
len is 54
54
len is 55
55
len is 56
56
len is 57
57
len is 58
58
len is 59
59
len is 60
60
len is 61
61
len is 62
62
len is 63
63
len is 64
64
len is 65
65
len is 66
66
len is 67
67
len is 68
68
len is 69
69
len is 70
70
len is 71
71
len is 72
72
len is 73
73
len is 74
74
len is 75
75
len is 76
76
len is 77
77
len is 78
78
le

ValueError: ignored

In [ ]:
finall_output


,input_id,output_id,input_text,output_text,chapter,tag1,tag2,-2,-1,0,1,2
0,0.0,5318.0,باید حقوق دیگران را رعایت کنیم,و قال ع من نصب نفسه للناس اماما فلیبدا بتعلیم ...,73,1,1,0,0,0,2,0
1,4.0,5564.0,نباید براساس ظاهر افراد را قضاوت کرد,و قال ع للظالم من الرجال ثلاث علامات یظلم من ف...,350,0,0,0,0,2,0,0
2,9.0,5563.0,باید قدردان دارایی‌های خودمان باشیم,و قال ع اشد الذنوب ما استهان به صاحبه,348,0,0,0,0,2,0,0
3,13.0,5563.0,باید بخشنده باشیم,و قال ع اشد الذنوب ما استهان به صاحبه,348,0,0,0,0,2,0,0
4,14.0,5414.0,باید حق‌جو وعدالت‌پیشه باشیم,و قال ع ما شککت فی الحق مذ اریته,184,2,1,0,0,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...
1445,33.0,3406.0,نباید در خوردن زیاده‌روی کرد,You can never be overdressed or overeducated,NaN,0,0,0,0,2,0,0
1446,36.0,3172.0,باید اهل یاری کردن یکدیگر بود,چون پرده براندازد عالم به سرانداز جایی که یقین...,NaN,0,0,0,0,2,0,0
1447,3.0,5310.0,باید خاک وطن را دوست داشت,و قال ع فقد الاحبه غربه,65,0,1,0,0,1,1,0
1448,16.0,5592.0,نباید از دیگران درخواست های متعدد داشت,و قال ع من طلب شیئا ناله او بعضه,386,0,-1,0,1,1,0,0


# Fleiss Kappa Score

In [ ]:
kapa = finall_output[["-2","-1","0","1","2"]]

In [ ]:
kapa

,-2,-1,0,1,2
0,0,0,0,2,0
1,0,0,2,0,0
2,0,0,2,0,0
3,0,0,2,0,0
4,0,0,0,1,1
...,...,...,...,...,...
1445,0,0,2,0,0
1446,0,0,2,0,0
1447,0,0,1,1,0
1448,0,1,1,0,0


In [ ]:

fleiss_kappa(kapa)

0.293470814941892

# NCDG Score For Each Model


In [ ]:
quotes_pair_labse =pd.read_csv("/content/quotes_pair.csv")


In [ ]:


def create_tag(model,k,path):
  input_table =pd.read_csv(path)
  # print(input_table)
  table = input_table.drop(['Unnamed: 0','input_text','output_text','input_vector','output_vector'], axis=1)
  columns = ["model","input_id","k","tag"]
  output = pd.DataFrame([], columns=columns)
  for i in range(len(table)):
    input_id = int(table.iloc[i]['input_id'])
    output_id = int(table.iloc[i]['output_id'])
    for j in range(len(finall_output)):
      input_id2 = int(finall_output.iloc[j]['input_id'])
      output_id2 = int(finall_output.iloc[j]['output_id'])
      if(input_id == input_id2 and output_id==output_id2):
        tag1 = finall_output.iloc[j]['tag1']
        tag2 = finall_output.iloc[j]['tag2']
        mean_tag = abs((abs(tag1) + abs(tag2)) / 2)
        output = output.append({'model':model,'input_id':table.iloc[i]["input_id"],'k':table.iloc[i]["k"],'tag':mean_tag}, ignore_index = True)
  true_score = []
  relevance_score = []
  for i in range(len(output)):
    if ( i%k == 0 ):
      x = [ output.iloc[i+j]["tag"] for j in range(k) ]
      # x = [ output.iloc[i-1]["tag"] , output.iloc[i]["tag"] ]
      # print(x)
      # print(type(x))
      # print(relevance_score)
      relevance_score.append(x)
      x.sort()
      true_score.append(x)
  return (ndcg_score(true_score, relevance_score))

In [ ]:
!mkdir labse 

In [ ]:
labse_quotes_pair_path = "/content/labse/quotes_pair.csv"
labse_nahj_pair_path = "/content/labse/nahj_pair.csv"
labse_hafez_pair_path = "/content/labse/hafez_pair.csv"
labse_poem_pair_path = "/content/labse/poem_pair.csv"
labse_shams_pair_path = "/content/labse/shams_pair.csv"
LaBSE_NDCG = [ create_tag("LaBSE",3,labse_poem_pair_path),
create_tag("LaBSE",2,labse_quotes_pair_path),
create_tag("LaBSE",2,labse_shams_pair_path),
create_tag("LaBSE",2,labse_hafez_pair_path)]

LaBSE_NDCG



[0.8, 0.8, 0.8, 0.78]

In [ ]:
!mkdir fast 

In [ ]:
fast_quotes_pair_path = "/content/fast/quotes_pair.csv"
fast_nahj_pair_path = "/content/fast/arabic_output_for_tag.csv"
fast_hafez_pair_path = "/content/fast/hafez_pair.csv"
fast_poem_pair_path = "/content/fast/poem_pair.csv"
fast_shams_pair_path = "/content/fast/shams_pair.csv"
Fast_Text_NDCG = [ create_tag("Fast_Text",3,fast_poem_pair_path),
create_tag("Fast_Text",2,fast_quotes_pair_path),
create_tag("Fast_Text",2,fast_shams_pair_path),
create_tag("Fast_Text",2,fast_hafez_pair_path)]

Fast_Text_NDCG



[0.38, 0.16, 0.2, 0.26]

In [ ]:
laser_all_pair_path = "/content/laser_pairs_out (2).csv"
laser_all_pair =pd.read_csv(laser_all_pair_path)

# labse_nahj_pair_path = "/content/nahj_pair.csv"
# labse_hafez_pair_path = "/content/hafez_pair.csv"
# labse_poem_pair_path = "/content/poem_pair.csv"
# labse_shams_pair_path = "/content/shams_pair.csv"
# LaBSE_NDCG = [ create_tag("LaBSE",3,labse_poem_pair_path),
# create_tag("LaBSE",2,labse_quotes_pair_path),
# create_tag("LaBSE",2,labse_shams_pair_path),
# create_tag("LaBSE",2,labse_hafez_pair_path)]

# LaBSE_NDCG
laser_all_pair
